# چک کردن کارت گرافیک

In [ ]:
!nvidia-smi

#دیتاست

پیکره خبری فارسی پرسیکا جمع آوری شده توسط دکتر اقبال زاده و همکاران

[لینک دانلود](https://sourceforge.net/projects/persica)

In [ ]:
!wget "https://sourceforge.net/projects/persica/files/persica.csv"

# نصب کردن کتابخانه هضم برای تمیز کردن پیکره
[گیت‌هاب](https://github.com/sobhe/hazm) 

[وب‌سایت](https://www.sobhe.ir/hazm/)

In [ ]:
!pip install -q hazm 

# پیکره

In [ ]:
import pandas as pd
from hazm.PersicaReader import PersicaReader

data = {
    "id": [],
    "title": [],
    "text": [],
    "date": [],
    "time": [],
    "category": [],
    "category2": []
}

for entry in PersicaReader('/content/persica.csv').docs():

    data['id'].append(entry['id'])
    data['title'].append(entry['title'])
    data['text'].append(entry['text'])
    data['date'].append(entry['date'])
    data['time'].append(entry['time'])
    data['category'].append(entry['category'])
    data['category2'].append(entry['category2'])

df = pd.DataFrame(data)
df.head()

## حذف کردن قسمت های اضافه

In [ ]:
df = df.replace("", float("NaN"))
df = df.dropna().reset_index(drop=True)
df.head()

In [ ]:
df = df[['title','text','category2']]
df.head()

## تبدیل کردن دسته بندی های فارسی به انگلیسی

In [ ]:
translation_dictionary = {
    "مذهبي" : "Religion",
    "آموزشي": "Education",
    "فقه و حقوق": "Law",
    "بهداشتي": "Health",
    "علمي": "Science",
    "تاريخي": "History",
    "ورزشي": "Sports",
    "اقتصادي": "Economy",
    "فرهنگي": "Culture",
    "سياسي": "Politics",
    "اجتماعي": "Social"
}
df["en_category"] = df.category2.apply(lambda x: translation_dictionary[x])
df.head()

## بررسی توزیع ورودی های هر کلاس

In [ ]:
import matplotlib.pyplot as plt

counts = df.en_category.value_counts()

labels = list(counts.keys())
sizes = list(counts.values)
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

## پیکره

### تعریف یک تابع برای تمیز کردن پیکره

In [ ]:
import hazm
normalizer = hazm.Normalizer(persian_numbers=True)
tokenizer = hazm.WordTokenizer()
stopwords = hazm.stopwords_list()

In [ ]:
def clean_text(sentence):
    sentence = normalizer.normalize(sentence)
    sentence = tokenizer.tokenize(sentence)
    return " ".join([word for word in sentence if word not in stopwords])

### استفاده از تابع بالا

In [ ]:
df = df.dropna()
df['cleaned_text'] = df["title"] + " " + df['text']
df['cleaned_text'] = df.cleaned_text.apply(clean_text)
df.head()

### اختصاص دادن یک عدد به هر کلاس

In [ ]:
labels = set(df.en_category)
print(f"labels: {labels}")

In [ ]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}
print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

In [ ]:
df['num_category'] = df.en_category.apply(lambda x: label2id[x])
df.head()

In [ ]:
df = df[['cleaned_text', 'num_category', 'en_category']]
df.head()

### بُر زدن پیکره

In [ ]:
df = df.sample(frac=1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=1)
train, valid = train_test_split(train, test_size=0.2, random_state=1)

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
print(f"training data: {train.shape}")
print(f"validation data: {valid.shape}")
print(f"test data: {test.shape}")

### تبدیل دیتافریم پانداس به دیتاست هاگینگ فیس

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train)
valid_dataset = Dataset.from_pandas(valid)
test_dataset = Dataset.from_pandas(test)

train_dataset

### تعریف یک تابع جهت آماده سازی دیتا برای آموزش

In [ ]:
MAX_LEN = 128

In [ ]:
import tensorflow as tf

def tensorify(dataset, tokenizer, max_len=MAX_LEN, tfds=True):
    dataset = dataset.map(lambda e: tokenizer(e['cleaned_text'], padding=True, max_length=max_len), batched=True)
    dataset.set_format(type='tensorflow', columns=['input_ids','token_type_ids', 'attention_mask','num_category'])
    features = {x: dataset[x].to_tensor(default_value=0, shape=[None, max_len]) for x in ['input_ids', 'token_type_ids', 'attention_mask']}
    if tfds:
        tfdataset = tf.data.Dataset.from_tensor_slices((features, dataset["num_category"]))
        return tfdataset
    else:
        return features, dataset["num_category"]

### آماده کردن توکنایزر پارس برت و تبدیل پیکره به بردار


In [ ]:
!pip install -q transformers 

In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = 'HooshvareLab/bert-fa-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_ds = tensorify(train_dataset, tokenizer)
valid_ds = tensorify(valid_dataset, tokenizer)
test_ds = tensorify(test_dataset, tokenizer)
xtest, ytest = tensorify(test_dataset, tokenizer, tfds=False)

### محاصبه کردن قدم ها و آماده کردن دیتاست

In [ ]:
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8

In [ ]:
train_steps = len(train) // TRAIN_BATCH_SIZE
valid_steps = len(valid) // VALID_BATCH_SIZE

train_steps, valid_steps

In [ ]:
train_dataset = train_ds.batch(TRAIN_BATCH_SIZE).repeat()
valid_dataset = valid_ds.batch(VALID_BATCH_SIZE).repeat()

# مدل

### تنظیم مدل

In [ ]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_NAME, **{
    'label2id': label2id,
    'id2label': id2label
})
print(config.to_json_string())

### تعریف تابع برای آماده کردن مدل

In [ ]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

def build_model():
    model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, config=config)
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
    return model

### ساخت مدل و آموزش آن

In [ ]:
EPOCHS = 2

In [ ]:
model = build_model()
r = model.fit(
            train_dataset,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            epochs=EPOCHS,
            verbose=1)

### بررسی روند آموزش

In [ ]:
acc = r.history['accuracy']
val_acc = r.history['val_accuracy']
loss = r.history['loss']
val_loss = r.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc', color="red")
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss', color="red")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## ارزیابی مدل

### Validation دقت

In [ ]:
import numpy as np

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

### Test دقت

In [ ]:
TEST_BATCH_SIZE = 8

In [ ]:
ev = model.evaluate(test_ds.batch(TEST_BATCH_SIZE))
print()
print(f'Evaluation: {ev}')

### پیش بینی بر روی دیتا های تست

In [ ]:
predictions = model.predict(xtest)
ypred = [ np.argmax(i) for i in predictions[0]]

### گزارش طبقه بندی

In [ ]:
from sklearn.metrics import classification_report, f1_score

class_names = id2label.values()
print(classification_report(ytest.numpy().tolist(), ypred, target_names=class_names))
print(f'F1: {f1_score(ytest.numpy().tolist(), ypred, average="weighted")}')

## ذخیره کردن مدل

In [ ]:
import os

OUTPUT_PATH = '/content/trained_model/parsbert_clf.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

model.save_pretrained(os.path.dirname(OUTPUT_PATH))

### لود کردن مدل ذخیره شده برای استفاده مجدد

In [ ]:
saved_model = TFBertForSequenceClassification.from_pretrained("/content/trained_model/")

In [ ]:
saved_model.summary()

# برای مطالعه بیشتر

[وبسایت هاگینگ‌فیس](https://huggingface.co/)

[داکیومنتیشن ترنسفرمرها](https://huggingface.co/transformers/)

[داکیومنتیشن دیتاستز](https://huggingface.co/docs/datasets/)

[داکیومنتیشن توکنایزر ها](https://huggingface.co/docs/tokenizers/python/latest/)

[مقاله پارس‌برت](https://arxiv.org/abs/2005.12515)

